# Data Dive 5
## Marketing Work for Food Delivery Service

### Kennidee Jones, Hayley Stanley, Bunker Brock, Nicholas Sartino

In [ ]:
# Modules

#!pip install missingno
#!pip install plotly-orca
#!pip install -U kaleido
#!pip install geopandas

In [ ]:
# Imports

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import missingno as msno
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import json

In [ ]:
# Read data from multiple sources and concatenate data frames
df_test = (pd.read_csv('test.csv', header = 0))
df_train = (pd.read_csv('train.csv',header = 0))

df = pd.concat([df_test,df_train],ignore_index=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# Get the count of NaN values for each column
na_counts = df.isna().sum()

print(na_counts)

In [ ]:
# Visualize missing values
msno.matrix(df)

In [ ]:
# Specify the columns of interest
columns_of_interest = ['Restaurant_latitude', 'Restaurant_longitude']

# Create an empty DataFrame to store rows with 0 values
zero_values_df = pd.DataFrame()

# Loop through columns and rows
for column in columns_of_interest:
    # Use boolean indexing to select rows where the value is 0
    zero_rows = df[df[column] == 0]
    
    # Concatenate the selected rows to the new DataFrame
    zero_values_df = pd.concat([zero_values_df, zero_rows], ignore_index=True)

# Display the new DataFrame
zero_values_df.head()

In [ ]:
# Drop these rows from the main dataframe because they make no sense

# Use boolean indexing to select rows where any of the specified columns have a value of 0
rows_to_drop = df[(df[columns_of_interest] == 0).any(axis=1)]

# Drop the selected rows from the original DataFrame
df = df.drop(rows_to_drop.index)

In [ ]:
# Clean up negative lat/long values
negative_lat_long_columns = ['Delivery_location_latitude', 'Delivery_location_longitude','Restaurant_latitude', 'Restaurant_longitude']

# Convert negative values to positive equivalents
df[negative_lat_long_columns] = df[negative_lat_long_columns].abs()

In [ ]:
# Create a feature for rows with no delivery lat/long and call it "Ate in Store"

# Specify the columns of interest
in_store_columns = ['Delivery_location_latitude', 'Delivery_location_longitude']

# Create a boolean mask for rows where both 'Delivery_location_latitude' and 'Delivery_location_longitude' are equal to 0
in_store_mask = (df[in_store_columns] < 70).all(axis=1)

# Create the 'In Store' column and set values based on the conditions
df['In Store'] = 'Delivery'  # Default value for all rows
df.loc[in_store_mask, 'In Store'] = 'In Store'

In [ ]:
# Using pd.isna()
nan_rows = df[pd.isna(df['Time_taken(min)'])]

nan_rows.head()

In [ ]:
# Using pd.notna()
not_nan_rows = df[pd.notna(df['Time_taken(min)'])]

not_nan_rows.head()

In [ ]:
df.head(100)

In [ ]:
# Clean up Weather Values
column_to_modify = 'Weatherconditions'

# Remove the string 'condition' from each value in the specified column
df[column_to_modify] = df[column_to_modify].str.replace('conditions ', '', regex=False)


In [ ]:
df.head()

In [ ]:
# Create a GeoDataFrame from the DataFrame by creating Point geometries
geometry = [Point(xy) for xy in zip(df['Delivery_location_longitude'], df['Delivery_location_latitude'])]
geo_df = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

# Plot the GeoDataFrame
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world.plot(figsize=(10, 6), color='white', edgecolor='black')

# Plot the GeoDataFrame on top
geo_df.plot(ax=ax, color='red', marker='o', markersize=50)

# Display the plot
plt.show()

In [ ]:
# Count the occurrences of each category in the 'In Store' column
traffic_counts = df['Type_of_vehicle'].value_counts()

# Create a bar chart
plt.bar(traffic_counts.index, traffic_counts.values, color=['blue', 'green'])

# Add labels and title
plt.xlabel('Delivery Type')
plt.ylabel('Count')
plt.title('Count of Deliveries by Vehicle Type')

# Show the plot
plt.show()

In [ ]:
# Count the occurrences of each category in the 'In Store' column
order_counts = df['Type_of_order'].value_counts()

# Create a bar chart
plt.bar(order_counts.index, order_counts.values, color=['blue', 'green'])

# Add labels and title
plt.xlabel('Order Type')
plt.ylabel('Count')
plt.title('Count of Deliveries by Order Type')

# Show the plot
plt.show()

In [ ]:
# Count the occurrences of each category in the 'Weatherconditions' column
weather_counts = df['Weatherconditions'].value_counts()

# Create a bar chart
plt.bar(weather_counts.index, weather_counts.values, color=['blue', 'green'])

# Add labels and title
plt.xlabel('Weather Conditions')
plt.ylabel('Count')
plt.title('Count of Deliveries by Weather Condition')
plt.xticks(rotation=90)

# Show the plot
plt.show()

In [ ]:
print(weather_counts)

In [ ]:
# Count the occurrences of each category in the 'Road_traffic_density' column
traffic_counts = df['Road_traffic_density'].value_counts()

# Create a bar chart
plt.bar(traffic_counts.index, traffic_counts.values, color=['blue', 'green'])

# Add labels and title
plt.xlabel('Traffic Conditions')
plt.ylabel('Count')
plt.title('Count of Deliveries by Traffic Conditions')

# Show the plot
plt.show()

In [ ]:
import os
print(os.getcwd())

In [ ]:
delivery = df.copy()

In [ ]:
fp = 'india_state_geo.json'
map_df = gpd.read_file(fp) 
map_df_copy = gpd.read_file(fp)
map_df.head()

In [ ]:
merged = map_df.set_index('geometry').join(delivery.set_index('State'))

In [ ]:
map_df.plot()

In [ ]:
# import plotly.express as px

# fig= px.scatter_geo(delivery, lat=delivery['Delivery_location_latitude'], lon = delivery['Delivery_location_longitude'])
# fig.update_geos(fitbounds='locations', visible=False)
# fig2=px.scatter_geo(delivery, lat=delivery['Restaurant_latitude'], lon=delivery['Restaurant_longitude'])
# fig.show()

In [ ]:
indian_states=json.load(open('india_state_geo.json'))

In [ ]:
f, ax = plt.subplots(figsize=(18, 14))

sns.lineplot(x=delivery['Delivery_person_Ratings'], y=delivery['Delivery_person_Age'] ,data=delivery, palette='magma')
plt.show()

In [ ]:
# Specify the columns of interest
columns_of_interest = ['Restaurant_latitude', 'Restaurant_longitude']

# Create an empty DataFrame to store rows with 0 values
zero_values_df = pd.DataFrame()

# Loop through columns and rows
for column in columns_of_interest:
    # Use boolean indexing to select rows where the value is 0
    zero_rows = delivery[delivery[column] == 0]
    
    # Concatenate the selected rows to the new DataFrame
    zero_values_df = pd.concat([zero_values_df, zero_rows], ignore_index=True)

# Display the new DataFrame
zero_values_df.head()

In [ ]:
#  Drop these rows from the main dataframe because they make no sense

# Use boolean indexing to select rows where any of the specified columns have a value of 0
rows_to_drop = delivery[(delivery[columns_of_interest] == 0).any(axis=1)]

# Drop the selected rows from the original DataFrame
delivery = delivery.drop(rows_to_drop.index)

In [ ]:
delivery['Delivery_person_Ratings'].isnull().sum()

In [ ]:

#delivery_copy=delivery.copy(deep=True)
#delivery_copy['Delivery_person_Ratings']= pd.to_numeric(delivery['Delivery_person_Ratings'])
#bins=[0,1,2,3,4,5]
#labels=['0-1','1-2','2-3','3-4','4-5']
#delivery_copy['Delivery_person_Ratings']=pd.cut(delivery_copy['Delivery_person_Ratings'], bins=bins, labels=labels)

In [ ]:
delivery['Delivery_person_Ratings'].dropna

In [ ]:
# customize title
delivery['Delivery_person_Ratings'] = delivery['Delivery_person_Ratings'].astype('float')

def categorize_Ratings(Delivery_person_Ratings): 
    if Delivery_person_Ratings >0.1 and Delivery_person_Ratings <=1:
        return "Extremely Low"
    elif Delivery_person_Ratings>1 and Delivery_person_Ratings <=2.5:
        return "Low"
    elif Delivery_person_Ratings>2.5 and Delivery_person_Ratings <=4:
        return "Medium"
    else:
        return "High"
    
delivery['rating_level'] = delivery['Delivery_person_Ratings'].apply(categorize_Ratings)
delivery.head(10)

In [ ]:
# Set up plot and styles
f, ax = plt.subplots(figsize=(12, 8))
plt.subplots_adjust(hspace=0.3, wspace=0.5)
sns.set_style('white')

rating_order = ['Extremely Low', 'Low', 'Medium', 'High']

# plot of the different subjectivity levels
sns.countplot(x="rating_level", data=delivery, order=rating_order, palette="magma")
# customize title
ax.set_title('Rating Levels', fontsize=12, weight='bold', style='italic', y=1)
ax.set_xlabel('', weight='bold', style='italic')
# customize y-axis label
ax.set_ylabel('Number of Ratings', weight='bold', style='italic')
# customize the y-axis label
ax.set_facecolor('black')
plt.show()

In [ ]:
delivery['Delivery_person_Age'] = delivery['Delivery_person_Age'].astype('float')
def categorize_Ratings(Delivery_person_Age): 
    if Delivery_person_Age >15 and Delivery_person_Age <=20:
        return "15-18"
    elif Delivery_person_Age>20 and Delivery_person_Age <=29:
        return "20-29"
    elif Delivery_person_Age>29 and Delivery_person_Age <=50:
        return "30-40"
    else:
        return "41 and up"
delivery['age_level'] = delivery['Delivery_person_Age'].apply(categorize_Ratings)
delivery.head()

In [ ]:
# Set up plot and styles
f, ax = plt.subplots(figsize=(12, 8))
plt.subplots_adjust(hspace=0.3, wspace=0.5)
sns.set_style('white')

rating_order = ['Extremely Low', 'Low', 'Medium', 'High']

# plot of the different subjectivity levels
sns.countplot(x="rating_level", data=delivery, order=rating_order, palette="magma", hue='Road_traffic_density')
# customize title
ax.set_title('Rating Levels', fontsize=12, weight='bold', style='italic', y=1)
ax.set_xlabel('', weight='bold', style='italic')
# customize y-axis label
ax.set_ylabel('Number of Ratings', weight='bold', style='italic')
# customize the y-axis label
ax.set_facecolor('black')
plt.show()

In [ ]:
# Set up plot and styles
f, ax = plt.subplots(figsize=(12, 8))
plt.subplots_adjust(hspace=0.3, wspace=0.5)
sns.set_style('white')

rating_order = ['Extremely Low', 'Low', 'Medium', 'High']

# plot of the different subjectivity levels
sns.countplot(x="rating_level", data=delivery, order=rating_order, palette="magma", hue='Type_of_vehicle')
# customize title
ax.set_title('Rating Levels', fontsize=12, weight='bold', style='italic', y=1)
ax.set_xlabel('', weight='bold', style='italic')
# customize y-axis label
ax.set_ylabel('Number of Ratings', weight='bold', style='italic')
# customize the y-axis label
ax.set_facecolor('black')
plt.show()

In [ ]:
# Set up plot and styles
f, ax = plt.subplots(figsize=(12, 8))
plt.subplots_adjust(hspace=0.3, wspace=0.5)
sns.set_style('white')

rating_order = ['Extremely Low', 'Low', 'Medium', 'High']

# plot of the different subjectivity levels
sns.countplot(x="rating_level", data=delivery, order=rating_order, palette="magma", hue='age_level')
# customize title
ax.set_title('Rating Levels', fontsize=12, weight='bold', style='italic', y=1)
ax.set_xlabel('', weight='bold', style='italic')
# customize y-axis label
ax.set_ylabel('Number of Ratings', weight='bold', style='italic')
# customize the y-axis label
ax.set_facecolor('black')
plt.show()

In [ ]:
delivery.describe()

In [ ]:
data = pd.read_csv('train.csv',na_values='NaN ')
data.isnull().sum()type.values.tolist()
time.values.tolist()
number_of_del.values.tolist()
ratings.values.tolist()
age.values.tolist()
weather.values.tolist()
traffic.values.tolist()
vehicle_cond.values.tolist()
type_order.values.tolist() 
type_veh.values.tolist() 

In [ ]:
time =data['Time_taken']
number_of_del = data['multiple_deliveries']
type = data['City']
ratings = data['Delivery_person_Ratings']
age = data['Delivery_person_Age']
weather = data['Weatherconditions']
traffic = data['Road_traffic_density']
vehicle_cond = data['Vehicle_condition']
type_order = data['Type_of_order']
type_veh = data['Type_of_vehicle']

In [ ]:
metropolitian_time = []
metropolitian_del = []
metropolitian_rate = []
for i in range(len(type)):
    if type[i] == "Metropolitian ":
        metropolitian_time.append(time[i])
        metropolitian_del.append(number_of_del[i])
        metropolitian_rate.append(ratings[i])
    else:
         continue
   

In [ ]:
urban_time = []
urban_del = []
urban_rate = []

for i in range(len(type)):
    if type[i] == 'Urban ':
        urban_time.append(time[i])
        urban_del.append(number_of_del[i])
        urban_rate.append(ratings[i])
    else:
        continue

In [ ]:
semi_urban_time = []
semi_urban_del = []
semi_urban_rate = []

for i in range(len(type)):
    if type[i] == 'Semi-Urban ':
        semi_urban_time.append(time[i])
        semi_urban_del.append(number_of_del[i])
        semi_urban_rate.append(ratings[i])
    else:
        continue

In [ ]:
df_metro = pd.DataFrame(metropolitian_time , columns = ['Time'])
df_metro.insert( 1, "Number of Deliveries", metropolitian_del, True)
df_metro.insert( 2, "Ratings", metropolitian_rate, True)
df_metro


In [ ]:
df_su = pd.DataFrame(semi_urban_time , columns = ['Time'])
df_su.insert( 1, "Number of Deliveries", semi_urban_del, True)
df_su.insert( 2, "Ratings", semi_urban_rate, True)
df_su

In [ ]:
df_u = pd.DataFrame(urban_time , columns = ['Time'])
df_u.insert( 1, "Number of Deliveries", urban_del, True)
df_u.insert( 2, "Ratings", urban_rate, True)
df_u['Number of Deliveries'].unique()

In [ ]:
df_u['Ratingas'].unique()

In [ ]:
df_u = df_u.dropna()

In [ ]:
df_su = df_su.dropna()

In [ ]:
df_metro = df_metro.dropna()

In [ ]:

p = sns.scatterplot(x=df_metro['Number of Deliveries'],y=df_metro['Ratings'],data=df_metro)
p.set_title('Metropolitian: Multiple Deliveries vs Ratings')

p.tick_params(labelsize=15)
p.set_title('Metropolitian: Multiple Deliveries vs Ratings', fontsize = 20, pad = 20)
p.set_xlabel("Number of Deliveries",fontsize=17)
p.set_ylabel('Ratings',fontsize=17)

f = p.get_figure()
f.savefig('metro del v rate.jpg', bbox_inches='tight')


In [ ]:
k = sns.scatterplot(x=df_su['Number of Deliveries'],y=df_su['Ratings'],data=df_su)
k.set_title('Semi-Urban: Multiple Deliveries vs Ratings')
f = k.get_figure()

k.tick_params(labelsize=15)
k.set_title('Semi-Urban: Multiple Deliveries vs Ratings', fontsize = 20, pad = 20)
k.set_xlabel("Number of Deliveries",fontsize=17)
k.set_ylabel('Ratings',fontsize=17)

f.savefig('semi del v rate.jpg', bbox_inches='tight')

In [ ]:
k = sns.scatterplot(x=df_u['Number of Deliveries'],y=df_u['Ratings'],data=df_u)
k.set_title('Urban: Multiple Deliveries vs Ratings')
f = k.get_figure()

k.tick_params(labelsize=15)
k.set_title('Urban: Multiple Deliveries vs Ratings', fontsize = 20, pad = 20)
k.set_xlabel("Number of Deliveries",fontsize=17)
k.set_ylabel('Ratings',fontsize=17)

f.savefig('urban del v rate.jpg', bbox_inches='tight')

In [ ]:
data['Delivery_person_Ratings'].unique()

In [ ]:
#look at the ouliers and what may be causing the low ratings Not multiple delivery like i thought 

metro6_time = []
metro6_nod = []
metro6_rate = []
metro6_age = []
metro6_weather = []
metro6_traffic = []
metro6_vc = []
metro6_to = []
metro6_tv = []
for i in range(len(type)):
    if type[i] == 'Metropolitian ' and ratings[i] == 6:
        metro6_time.append(time[i])
        metro6_nod.append(number_of_del[i])
        metro6_rate.append(ratings[i])
        metro6_age.append(age[i])
        metro6_weather.append(weather[i])
        metro6_traffic.append(traffic[i])
        metro6_vc.append(vehicle_cond[i])
        metro6_to.append(type_order[i])
        metro6_tv.append(type_veh[i])
        print(i)
    else:
        continue

In [ ]:
a = np.array(metro6_time)
average = np.mean(metro6_time)
print(average)

In [ ]:
df_metro6 = pd.DataFrame(metro6_time , columns = ['Time'])
df_metro6.insert( 1, "Number of Deliveries", metro6_nod, True)
df_metro6.insert( 2, "Ratings", metro6_rate, True)
df_metro6.insert( 3, "Age", metro6_age, True)
df_metro6.insert( 4, "Weather", metro6_weather, True)
df_metro6.insert( 5, "Traffic", metro6_traffic, True)
df_metro6.insert( 6, "Vehicle Condition", metro6_vc, True)
df_metro6.insert( 7, "Type of Order", metro6_to, True)
df_metro6.insert( 8, "Type of Vehicle", metro6_tv, True)
df_metro6

In [ ]:
metro1_time = []
metro1_nod = []
metro1_rate = []
metro1_age = []
metro1_weather = []
metro1_traffic = []
metro1_vc = []
metro1_to = []
metro1_tv = []
for i in range(len(type)):
    if type[i] == 'Metropolitian ' and ratings[i] == 1:
        metro1_time.append(time[i])
        metro1_nod.append(number_of_del[i])
        metro1_rate.append(ratings[i])
        metro1_age.append(age[i])
        metro1_weather.append(weather[i])
        metro1_traffic.append(traffic[i])
        metro1_vc.append(vehicle_cond[i])
        metro1_to.append(type_order[i])
        metro1_tv.append(type_veh[i])
        print(i)
    else:
        continue

In [ ]:
a = np.array(metro1_time)
average = np.mean(metro1_time)
print(average)

In [ ]:
df_metro1 = pd.DataFrame(metro1_time , columns = ['Time'])
df_metro1.insert( 1, "Number of Deliveries", metro1_nod, True)
df_metro1.insert( 2, "Ratings", metro1_rate, True)
df_metro1.insert( 3, "Age", metro1_age, True)
df_metro1.insert( 4, "Weather", metro1_weather, True)
df_metro1.insert( 5, "Traffic", metro1_traffic, True)
df_metro1.insert( 6, "Vehicle Condition", metro1_vc, True)
df_metro1.insert( 7, "Type of Order", metro1_to, True)
df_metro1.insert( 8, "Type of Vehicle", metro1_tv, True)
df_metro1

In [ ]:
p = sns.countplot(data = df_metro6,x = "Type of Order")

p.tick_params(labelsize=15)
p.set_title('Metropolitian Rating 6: Order Type', fontsize = 20, pad = 20)
p.set_xlabel("Type of Order",fontsize=17)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('metro6 ot.jpg', bbox_inches='tight')



In [ ]:
p = sns.countplot(data = df_metro1,x = "Type of Order")

p.tick_params(labelsize=15)
p.set_title('Metropolitian Rating 1: Order Type', fontsize = 20, pad = 20)
p.set_xlabel("Type of Order",fontsize=17)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('metro1 ot.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_metro6,x = "Type of Vehicle")

p.tick_params(labelsize=15)
p.set_title('Metropolitian Rating 6: Vehicle Type', fontsize = 20, pad = 20)
p.set_xlabel("Type of Vehicle",fontsize=17)
p.set_xticklabels(p.get_xticklabels(), rotation = 15)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('metro6 vt.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_metro1,x = "Type of Vehicle")

p.tick_params(labelsize=15)
p.set_title('Metropolitian Rating 1: Vehicle Type', fontsize = 20, pad = 20)
p.set_xlabel("Type of Vehicle",fontsize=17)
p.set_xticklabels(p.get_xticklabels(), rotation = 15)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('metro1 vt.jpg', bbox_inches='tight')

In [ ]:
u6_time = []
u6_nod = []
u6_rate = []
u6_age = []
u6_weather = []
u6_traffic = []
u6_vc = []
u6_to = []
u6_tv = []
for i in range(len(type)):
    if type[i] == 'Metropolitian ' and ratings[i] == 6:
        u6_time.append(time[i])
        u6_nod.append(number_of_del[i])
        u6_rate.append(ratings[i])
        u6_age.append(age[i])
        u6_weather.append(weather[i])
        u6_traffic.append(traffic[i])
        u6_vc.append(vehicle_cond[i])
        u6_to.append(type_order[i])
        u6_tv.append(type_veh[i])
        print(i)
    else:
        continue

In [ ]:
a = np.array(u6_time)
average = np.mean(u6_time)
print(average)

In [ ]:
df_u6 = pd.DataFrame(u6_time , columns = ['Time'])
df_u6.insert( 1, "Number of Deliveries", u6_nod, True)
df_u6.insert( 2, "Ratings", u6_rate, True)
df_u6.insert( 3, "Age", u6_age, True)
df_u6.insert( 4, "Weather", u6_weather, True)
df_u6.insert( 5, "Traffic", u6_traffic, True)
df_u6.insert( 6, "Vehicle Condition", u6_vc, True)
df_u6.insert( 7, "Type of Order", u6_to, True)
df_u6.insert( 8, "Type of Vehicle", u6_tv, True)
df_u6

In [ ]:
u1_time = []
u1_nod = []
u1_rate = []
u1_age = []
u1_weather = []
u1_traffic = []
u1_vc = []
u1_to = []
u1_tv = []
for i in range(len(type)):
    if type[i] == 'Metropolitian ' and ratings[i] == 1:
        u1_time.append(time[i])
        u1_nod.append(number_of_del[i])
        u1_rate.append(ratings[i])
        u1_age.append(age[i])
        u1_weather.append(weather[i])
        u1_traffic.append(traffic[i])
        u1_vc.append(vehicle_cond[i])
        u1_to.append(type_order[i])
        u1_tv.append(type_veh[i])
        print(i)
    else:
        continue

In [ ]:
a = np.array(u6_tim)
average = np.mean(u6_time)
print(average)

In [ ]:
df_u1 = pd.DataFrame(u1_time , columns = ['Time'])
df_u1.insert( 1, "Number of Deliveries", u1_nod, True)
df_u1.insert( 2, "Ratings", u1_rate, True)
df_u1.insert( 3, "Age", u1_age, True)
df_u1.insert( 4, "Weather", u1_weather, True)
df_u1.insert( 5, "Traffic", u1_traffic, True)
df_u1.insert( 6, "Vehicle Condition", u1_vc, True)
df_u1.insert( 7, "Type of Order", u1_to, True)
df_u1.insert( 8, "Type of Vehicle", u1_tv, True)
df_u1

In [ ]:
p = sns.countplot(data = df_u6,x = "Type of Order")

p.tick_params(labelsize=15)
p.set_title('Urban Rating 6: Order Type', fontsize = 20, pad = 20)
p.set_xlabel("Type of Order",fontsize=17)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('u6 ot.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_u1,x = "Type of Order")

p.tick_params(labelsize=15)
p.set_title('Urban Rating 1: Order Type', fontsize = 20, pad = 20)
p.set_xlabel("Type of Order",fontsize=17)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('u1 ot.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_u6,x = "Type of Vehicle")

p.tick_params(labelsize=15)
p.set_title('Urban Rating 6: Vehicle Type', fontsize = 20, pad = 20)
p.set_xlabel("Type of Vehicle",fontsize=17)
p.set_xticklabels(p.get_xticklabels(), rotation = 15)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('u6 vt.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_u1,x = "Type of Vehicle")

p.tick_params(labelsize=15)
p.set_title('Urban Rating 1: Vehicle Type', fontsize = 20, pad = 20)
p.set_xlabel("Type of Vehicle",fontsize=17)
p.set_xticklabels(p.get_xticklabels(), rotation = 15)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('u1 vt.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_metro6,x = "Age")

p.tick_params(labelsize=15)
p.set_title('Metro Rating 6: Age', fontsize = 20, pad = 20)
p.set_xlabel("Age",fontsize=17)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('metro6 age.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_metro1,x = "Age")

p.tick_params(labelsize=15)
p.set_title('Metro Rating 1: Age', fontsize = 20, pad = 20)
p.set_xlabel("Age",fontsize=17)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('metro1 age.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_u6,x = "Age")

p.tick_params(labelsize=15)
p.set_title('Urban Rating 6: Age', fontsize = 20, pad = 20)
p.set_xlabel("Age",fontsize=17)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('u6 age.jpg', bbox_inches='tight')

In [ ]:
p = sns.countplot(data = df_u1,x = "Age")

p.tick_params(labelsize=15)
p.set_title('Urban Rating 1: Age', fontsize = 20, pad = 20)
p.set_xlabel("Age",fontsize=17)
p.set_ylabel('Count',fontsize=17)

f = p.get_figure()
f.savefig('u1 age.jpg', bbox_inches='tight')